In [ ]:
# default_exp audio.learner

# Audio Learner
> Learner which knows to stack `FlattenTransform`

In [ ]:
#export
from __future__ import annotations

from fastcore.dispatch import retain_type

from fastai.callback.core import Callback
from fastai.callback.fp16 import MixedPrecision
from fastai.learner import Learner, defaults
from fastai.optimizer import Adam

from fastxtend.audio.data import MelSpectrogram, Spectrogram
from fastxtend.imports import *

In [ ]:
#export
class DetupleCallback(Callback):
    ""
    order = MixedPrecision.order-1
    def before_batch(self):
        xb = L(self.xb)
        idx = xb.argwhere(lambda x: isinstance(x, tuple))
        xb[i] = xb[i].map(lambda x: torch.cat(x, dim=1))
        for i in idx:
            xb[i] = retain_type(torch.cat(xb[i], dim=1), xb[i][0])
        self.learn.xb = tuple(xb)

In [ ]:
#export
class AudioLearner(Learner):
    "An Audio specific Learner that knows how to handel"
    def __init__(self, dls, model, loss_func=None, opt_func=Adam, lr=defaults.lr, splitter=trainable_params, cbs=None,
                metrics=None, path=None, model_dir='models', wd=None, wd_bn_bias=False, train_bn=True,
                moms=(0.95,0.85,0.95)):

        detuple = False
        for i in range(len(dls.train.after_batch.fs)):
            if not detuple and isinstance(dls.train.after_batch[i], (Spectrogram, MelSpectrogram)):
                detuple = is_listy(dls.train.after_batch[i].n_fft)
        
        if detuple:
            if cbs is None: cbs = DetupleCallback()
            else: cbs = L(cbs) + L(DetupleCallback())

        return Learner(dls=dls, model=model, loss_func=loss_func, opt_func=opt_func, lr=lr, splitter=splitter, cbs=cbs,
                       metrics=metrics, path=path, model_dir=model_dir, wd=wd, wd_bn_bias=wd_bn_bias, train_bn=train_bn,
                       moms=moms)